In [ ]:
! git clone https://github.com/Yang-Hyun-Jun/RLPortfolio-DirichletOnp-COLAB.git

Cloning into 'RLPortfolio-Dirichlet-COLAB'...
remote: Enumerating objects: 130, done.
remote: Counting objects: 100% (130/130), done.
remote: Compressing objects: 100% (94/94), done.
remote: Total 130 (delta 68), reused 97 (delta 35), pack-reused 0
Receiving objects: 100% (130/130), 563.04 KiB | 11.49 MiB/s, done.
Resolving deltas: 100% (68/68), done.


In [ ]:
! pip install mplfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 70 kB 3.7 MB/s 


In [ ]:
import sys
import os
import torch

In [ ]:
sys.path.insert(0, "/content/RLPortfolio-DirichletOnp-COLAB")

In [ ]:
os.makedirs("/content/Metrics", exist_ok=True)
os.makedirs("/content/Models", exist_ok=True)

In [ ]:
import DataManager
import utils

from Learner import DIRILearner
from Test import DIRITester

In [ ]:
# Data Load

stock_list = ["HA", "WBA", "INCY", "BIDU", "TCOM", "AAPL", "COST"]
path_list = []

for stock_name in stock_list:
  path = utils.SAVE_DIR + "/RLPortfolio-DirichletOnp-COLAB/Data/" + stock_name
  path_list.append(path)

train_data, test_data = DataManager.get_data_tensor(path_list,
                                                    train_date_start="2014-06-02",
                                                    train_date_end="2019-12-31",
                                                    test_date_start="2020-01-02",
                                                    test_date_end="2021-12-31")

-------------------------------------
학습 데이터 시작 거래일:2014-06-02
학습 데이터 마지막 거래일:2019-12-31
테스트 데이터 시작 거래일:2020-01-02
테스트 데이터 마지막 거래일:2021-12-31
-------------------------------------


In [ ]:
""""
K: 거래 종목 수
lr: learing rate (default 1e-5)
tau: soft target update (default 0.005)
delta: hold action threshold (default 0.005)
discount_factor: gamma (default 0.9)
num_episode: episode (default 50)
balance: 초기 투자금 (default 12000 달러) 
batch_size: 배치 사이즈 (default 30)
memory_size: 버퍼 사이즈 (default 100)
min_trading_price: 최소 거래 금액 (default 0달러)
max_trading_price: 최대 거래 금액 (default 400달러) 
repre: 테스트시 대푯값 ("mean" or "mode")
"""

# Hyperparameters
K = len(stock_list)
lr = 1e-5
tau = 0.005
delta = 0.005
discount_factor = 0.9
num_episode = 50
balance = 30000
batch_size = 30
memory_size = 100
min_trading_price = 0
max_trading_price = 1000
repre = "mean"

# paramter1 for training
parameters1 = {"lr":lr, 
               "tau":tau,
               "K":K,
               "delta":delta, "cost":0.0,
               "chart_data":train_data,
               "discount_factor":discount_factor,
               "min_trading_price":min_trading_price,
               "max_trading_price":max_trading_price,
               "batch_size":batch_size,
               "memory_size":memory_size}

# parameter2 : for test
parameters2 = {"K":K,
               "delta":delta,
               "balance":balance, "cost":0.0025,
               "min_trading_price":min_trading_price,
               "max_trading_price":max_trading_price,
               "test_data":test_data,
               "repre":repre,
               "holding":True} 

In [ ]:
# 학습 수행
learner = DIRILearner(**parameters1)
learner.run(num_episode=num_episode, balance=balance)
learner.save_model(critic_path=utils.SAVE_DIR + "/Models" + "/DirichletPortfolio_critic.pth",
                   actor_path=utils.SAVE_DIR + "/Models" + "/DirichletPortfolio_actor.pth",
                   score_net_path=utils.SAVE_DIR + "/Models" + "/DirichletPortfolio_score.pth")

In [ ]:
# 테스트 수행
tester = DIRITester(**parameters2)
tester.run() 

In [ ]:
! zip -r /content/Metrics.zip /content

In [ ]:
from google.colab import files
files.download("/content/Metrics.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>